In [3]:
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [4]:
%%cuda
#include<stdio.h>
int main()
{
int devcount;
cudaGetDeviceCount(&devcount);
printf("Device count:%d\n",devcount);
printf("hello");
for (int i = 0; i < devcount; ++i)
{
// Get device properties
printf("\nCUDA Device #%d\n", i);
cudaDeviceProp devProp;
cudaGetDeviceProperties(&devProp, i);
printf("Name:%s\n", devProp.name);
printf("Compute capability: %d.%d\n",devProp.major ,devProp.minor);
printf("Warp Size %d\n",devProp.warpSize);
printf("Total global memory:%u bytes\n",devProp.totalGlobalMem);
printf("Total shared memory per block: %u bytes\n", devProp.sharedMemPerBlock);
printf("Total registers per block : %d\n",devProp.regsPerBlock);
printf("Clock rate: %d khz\n",devProp.clockRate);
printf("Maximum threads per block:%d\n", devProp.maxThreadsPerBlock);
for (int i = 0; i < 3; ++i)
printf("Maximum dimension %d of block: %d\n", i, devProp.maxThreadsDim[i]);
for (int i = 0; i <= 2; ++i)
printf("Maximum dimension %d of grid: %d\n", i, devProp.maxGridSize[i]);
printf("Number of multiprocessors:%d\n", devProp.multiProcessorCount);
printf("max threads per multiprocessor: %d\n",devProp.maxThreadsPerMultiProcessor);
}
return 0;
}


Device count:1
hello
CUDA Device #0
Name:Tesla T4
Compute capability: 7.5
Warp Size 32
Total global memory:2950758400 bytes
Total shared memory per block: 49152 bytes
Total registers per block : 65536
Clock rate: 1590000 khz
Maximum threads per block:1024
Maximum dimension 0 of block: 1024
Maximum dimension 1 of block: 1024
Maximum dimension 2 of block: 64
Maximum dimension 0 of grid: 2147483647
Maximum dimension 1 of grid: 65535
Maximum dimension 2 of grid: 65535
Number of multiprocessors:40
max threads per multiprocessor: 1024



In [6]:
%%cuda
#include<stdio.h>
#include<cuda.h>
__global__ void helloworld(void)
{
printf("Hello World from GPU\n");
}
int main() {
helloworld<<<1,10>>>();
printf("Hello World\n");
return 0;
}

Hello World



In [7]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/time.h>

__global__ void vecAdd(double *a, double *b, double *c, int n)
{
// Get global thread
int id = blockIdx.x*blockDim.x+threadIdx.x;
// Do not go out of bounds
if (id < n)
c[id] = a[id] + b[id];
}
int main(int argc, char* argv[] )
{
// Size of vectors
int n = 100;
//time varibales
struct timeval t1, t2;
// Host input vectors
double *h_a, *h_b;
//Host output vector
double *h_c;
// Device input vectors
double *d_a, *d_b;
//Device output vector
double *d_c;
// Size, in bytes, of each vector
size_t bytes = n*sizeof(double);
// Allocate memory for each vector on host
h_a = (double*)malloc(bytes);
h_b = (double*)malloc(bytes);
h_c = (double*)malloc(bytes);
// Allocate memory for each vector on GPU
cudaMalloc(&d_a, bytes);
cudaMalloc(&d_b, bytes);
cudaMalloc(&d_c, bytes);
int i;
// Initialize vectors on host
for( i = 0; i < n; i++ ) {
h_a[i] = i+1;
h_b[i] = i+1;
}
// Copy host vectors to device
cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);

gettimeofday(&t1, 0);

// Execute the kernel
vecAdd<<<1,100>>>(d_a, d_b, d_c, n);
cudaDeviceSynchronize();
gettimeofday(&t2, 0);

// Copy array back to host
cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );

for(i=0; i<n; i=i+10)
printf("c[%d]=%f\n",i,h_c[i]);
double time = (1000000.0*(t2.tv_sec-t1.tv_sec) + t2.tv_usec-t1.tv_usec)/1000.0;

printf("Time to generate: %3.10f ms \n", time);

// Release device memory
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
// Release host memory
free(h_a);
free(h_b);
free(h_c);
return 0;
}

c[0]=2.000000
c[10]=22.000000
c[20]=42.000000
c[30]=62.000000
c[40]=82.000000
c[50]=102.000000
c[60]=122.000000
c[70]=142.000000
c[80]=162.000000
c[90]=182.000000
Time to generate: 36.2510000000 ms 



In [8]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/time.h>

__global__ void vecAdd(double *a, double *b, double *c, int n)
{
// Get global thread
int id = blockIdx.x*blockDim.x+threadIdx.x;
// Do not go out of bounds
if (id < n)
c[id] = a[id] + b[id];
}
int main(int argc, char* argv[] )
{
// Size of vectors
int n = 100;
//time varibales
struct timeval t1, t2;
// Host input vectors
double *h_a, *h_b;
//Host output vector
double *h_c;
// Device input vectors
double *d_a, *d_b;
//Device output vector
double *d_c;
// Size, in bytes, of each vector
size_t bytes = n*sizeof(double);
// Allocate memory for each vector on host
h_a = (double*)malloc(bytes);
h_b = (double*)malloc(bytes);
h_c = (double*)malloc(bytes);
// Allocate memory for each vector on GPU
cudaMalloc(&d_a, bytes);
cudaMalloc(&d_b, bytes);
cudaMalloc(&d_c, bytes);
int i;
// Initialize vectors on host
for( i = 0; i < n; i++ ) {
h_a[i] = i+1;
h_b[i] = i+1;
}
// Copy host vectors to device
cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);

gettimeofday(&t1, 0);

// Execute the kernel
vecAdd<<<1,50>>>(d_a, d_b, d_c, n);
cudaDeviceSynchronize();
gettimeofday(&t2, 0);

// Copy array back to host
cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );

for(i=0; i<n; i=i+10)
printf("c[%d]=%f\n",i,h_c[i]);
double time = (1000000.0*(t2.tv_sec-t1.tv_sec) + t2.tv_usec-t1.tv_usec)/1000.0;

printf("Time to generate: %3.10f ms \n", time);

// Release device memory
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
// Release host memory
free(h_a);
free(h_b);
free(h_c);
return 0;
}

c[0]=2.000000
c[10]=22.000000
c[20]=42.000000
c[30]=62.000000
c[40]=82.000000
c[50]=0.000000
c[60]=0.000000
c[70]=0.000000
c[80]=0.000000
c[90]=0.000000
Time to generate: 0.2110000000 ms 



In [9]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/time.h>

__global__ void vecAdd(double *a, double *b, double *c, int n)
{
// Get global thread
int id = blockIdx.x*blockDim.x+threadIdx.x;
// Do not go out of bounds
if (id < n)
c[id] = a[id] + b[id];
}
int main(int argc, char* argv[] )
{
// Size of vectors
int n = 100;
//time varibales
struct timeval t1, t2;
// Host input vectors
double *h_a, *h_b;
//Host output vector
double *h_c;
// Device input vectors
double *d_a, *d_b;
//Device output vector
double *d_c;
// Size, in bytes, of each vector
size_t bytes = n*sizeof(double);
// Allocate memory for each vector on host
h_a = (double*)malloc(bytes);
h_b = (double*)malloc(bytes);
h_c = (double*)malloc(bytes);
// Allocate memory for each vector on GPU
cudaMalloc(&d_a, bytes);
cudaMalloc(&d_b, bytes);
cudaMalloc(&d_c, bytes);
int i;
// Initialize vectors on host
for( i = 0; i < n; i++ ) {
h_a[i] = i+1;
h_b[i] = i+1;
}
// Copy host vectors to device
cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);

gettimeofday(&t1, 0);

// Execute the kernel
vecAdd<<<2,50>>>(d_a, d_b, d_c, n);
cudaDeviceSynchronize();
gettimeofday(&t2, 0);

// Copy array back to host
cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );

for(i=0; i<n; i=i+10)
printf("c[%d]=%f\n",i,h_c[i]);
double time = (1000000.0*(t2.tv_sec-t1.tv_sec) + t2.tv_usec-t1.tv_usec)/1000.0;

printf("Time to generate: %3.10f ms \n", time);

// Release device memory
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
// Release host memory
free(h_a);
free(h_b);
free(h_c);
return 0;
}

c[0]=2.000000
c[10]=22.000000
c[20]=42.000000
c[30]=62.000000
c[40]=82.000000
c[50]=102.000000
c[60]=122.000000
c[70]=142.000000
c[80]=162.000000
c[90]=182.000000
Time to generate: 0.2480000000 ms 

